In [1]:
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import itertools
import pandas as pd
# import corona
import numpy as np

# Covid Texting/Email
Inspired by this [blog post](https://towardsdatascience.com/how-to-track-coronavirus-with-python-a5320b778c8e)

In [3]:
class Coronavirus():
  def __init__(self, link):
    self.driver = webdriver.Chrome()
    self.driver.get(link)

In [ ]:
# bot_world = Coronavirus('https://www.worldometers.info/coronavirus/')
# bot_us = Coronavirus('https://www.worldometers.info/coronavirus/country/us/')
# bot_la = Coronavirus('http://ldh.la.gov/Coronavirus/')

## Louisiana Parishes

In [42]:
bot_la = Coronavirus('https://www.arcgis.com/apps/opsdashboard/index.html#/69b726e2b82e408f89c3a54f96e8f776')


In [48]:
lines = bot_la.driver.find_elements_by_xpath('//*[@id="ember56"]/div/nav/span/div/div/p[2]/strong')
lines = [line.get_attribute("innerHTML") for line in lines]


In [49]:
# https://stackoverflow.com/questions/1624883/alternative-way-to-split-a-list-into-groups-of-n
def grouper(n, iterable, fillvalue=None):
    "grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)

In [50]:
grouped_list = list(grouper(3, lines))

In [51]:
df = pd.DataFrame(grouped_list, columns =['Parish', 'Cases', 'Deaths']) 

In [73]:
df

,Parish,Cases,Deaths
0,Orleans,451,15
1,Jefferson,184,3
2,Catahoula,1,1
3,St. James,1,1
4,St. Tammany,25,0
...,...,...,...
60,Vermilion,0,0
61,Vernon,0,0
62,West Carroll,0,0
63,West Feliciana,0,0


In [59]:
parish_lens = [len(p) for p in df.Parish.values]

In [65]:
max_val = max(parish_lens)


In [72]:
df['Parish'] = ['{:26}'.format(p) for p in df.Parish.values]


In [70]:
['{:26}'.format(p) for p in df.Parish.values]


['Orleans                   ',
 'Jefferson                 ',
 'Catahoula                 ',
 'St. James                 ',
 'St. Tammany               ',
 'East Baton Rouge          ',
 'Ascension                 ',
 'Caddo                     ',
 'St. Bernard               ',
 'Lafourche                 ',
 'Terrebonne                ',
 'Parish Under Investigation',
 'St. John the Baptist      ',
 'St. Charles               ',
 'Lafayette                 ',
 'Bossier                   ',
 'Plaquemines               ',
 'Calcasieu                 ',
 'Ouachita                  ',
 'Rapides                   ',
 'St. Landry                ',
 'Tangipahoa                ',
 'West Baton Rouge          ',
 'De Soto                   ',
 'Evangeline                ',
 'Iberia                    ',
 'Iberville                 ',
 'Livingston                ',
 'Washington                ',
 'Acadia                    ',
 'Assumption                ',
 'Avoyelles                 ',
 'Beaure

## US States

In [215]:
bot_us = corona.get_us_bot()


In [66]:
df_us.to_csv('us_states_df.csv')

In [132]:
us_data = pd.read_csv('us_states_df.csv')
us_data.drop(columns="Unnamed: 0", inplace=True)
us_data.State.replace("<strong>Total:</strong>", "Total", inplace=True)

total = us_data[53:54]
# us_data[0:51]#.sort_values(by='State')


In [127]:
# this is for from the dataframe
us_data.NewCases = us_data.NewCases.str.replace("+","")
us_data.NewCases = us_data.NewCases.str.replace(",","").fillna(0).astype(int)

us_data.TotalCases = us_data.TotalCases.str.replace(",","").fillna(0).astype(int)
us_data.TotalDeaths = us_data.TotalDeaths.fillna(0).astype(int)
us_data.NewDeaths = us_data.NewDeaths.fillna(0).astype(int)

us_data["CasePercentInc"] = us_data.NewCases.str.replace(",","").fillna(0).astype(int) / (us_data.TotalCases.str.replace(",","").fillna(0).astype(int) - us_data.NewCases.str.replace(",","").fillna(0).astype(int))
us_data["DeathPercentInc"] = us_data.NewDeaths / (us_data.TotalDeaths - us_data.NewDeaths)


In [72]:
total

,State,TotalCases,NewCases,TotalDeaths,NewDeaths
53,Total,"52,447","8,713",674.0,121.0


In [60]:
# pd.set_option('display.max_rows', us_data.shape[0]+1)

In [220]:
# this is for directly from the source
df_us = corona.get_states_df(bot_us)

us_data = df_us.copy()
us_data.State.replace("<strong>Total:</strong>", "Total", inplace=True)
total = us_data[53:54]
us_data = us_data[0:51]#.sort_values(by='State')

us_data.NewCases = us_data.NewCases.str.replace("+","")
us_data.NewCases.str.replace(",","").fillna(0)
us_data.replace(r'^\s*$', np.nan, regex=True, inplace=True)
us_data["CasePercentInc"] = us_data.NewCases.str.replace(",","").fillna(0).astype(int) / (us_data.TotalCases.str.replace(",","").fillna(0).astype(int) - us_data.NewCases.str.replace(",","").fillna(0).astype(int))
us_data["DeathPercentInc"] = us_data.NewDeaths.fillna(0).astype(int) / (us_data.TotalDeaths.fillna(0).astype(int) - us_data.NewDeaths.fillna(0).astype(int))
                                                                              
   

In [182]:

"{:.1f}".format(us_data.CasePercentInc.values[0] * 100)

'22.9'

In [224]:
m = corona.get_us_state(bot_us, 'montana')

      State TotalCases NewCases TotalDeaths NewDeaths stripped_state
45  Montana         46       +1                              montana


In [211]:
m.State.dtypes

dtype('O')

In [212]:
m.NewDeaths.fillna(0)

45    
Name: NewDeaths, dtype: object

In [225]:
m

,State,TotalCases,NewCases,TotalDeaths,NewDeaths,stripped_state
45,Montana,46,+1,,,montana


In [246]:
def get_states_df(bot_us):
	states = bot_us.driver.find_elements_by_xpath("//tbody/tr/td")
	lines = [" ".join(state.get_attribute("innerHTML").split()) for state in states]
	grouped_list = list(corona.grouper(7, lines))
	df = pd.DataFrame(grouped_list).iloc[:,0:5]
	df.columns =['State', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths']
	df.State.replace("<strong>Total:</strong>", "Total", inplace=True)
	return df[0:54]

In [260]:
df = get_states_df(bot_us)


In [261]:
df['stripped_state'] = [corona.clean_text(p) for p in df.State.values]
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df.NewCases = df.NewCases.str.replace("+","").fillna(0)
df.NewDeaths = df.NewDeaths.str.replace("+","").fillna(0)


In [262]:
df.NewCases.str.replace(",","").fillna(0)#.astype(int)

0     4790
1      831
2      361
3        0
4      463
5      250
6      185
7      216
8      382
9      223
10     207
11      20
12       0
13      52
14     203
15     122
16      49
17      41
18     106
19      61
20      43
21      92
22      71
23      41
24      36
25      33
26      27
27      59
28      21
29      19
30      18
31      33
32       0
33      19
34      18
35      11
36      25
37       0
38      16
39      20
40      23
41       0
42      34
43       0
44      11
45       1
46       4
47       2
48       2
49       4
50       4
51       0
52       0
53    9249
Name: NewCases, dtype: object

In [263]:
state = "montana"

In [264]:
df['stripped_state'] = [corona.clean_text(p) for p in df.State.values]
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df.NewCases = df.NewCases.str.replace("+","").fillna(0)
df.NewDeaths = df.NewDeaths.str.replace("+","").fillna(0)
df.TotalDeaths = df.TotalDeaths.fillna(0)

df["CasePercentInc"] = df.NewCases.str.replace(",","").fillna(0).astype(int) / (df.TotalCases.str.replace(",","").fillna(0).astype(int) - df.NewCases.str.replace(",","").fillna(0).astype(int))
df["DeathPercentInc"] = df.NewDeaths.fillna(0).astype(int) / (df.TotalDeaths.fillna(0).astype(int) - df.NewDeaths.fillna(0).astype(int))

state = df[df.stripped_state == state]

In [265]:
state_name = state.State.values[0]
total_cases = state.TotalCases.values[0]
new_cases = state.NewCases.values[0]  or 0
case_percent = str("{:.1f}".format(state.CasePercentInc.values[0] * 100)) + "%"
total_deaths = state.TotalDeaths.values[0] or 0
new_deaths = state.NewDeaths.values[0] or 0
death_percent = str("{:.1f}".format(state.DeathPercentInc.values[0] * 100)) + "%"



In [266]:
state

,State,TotalCases,NewCases,TotalDeaths,NewDeaths,stripped_state,CasePercentInc,DeathPercentInc
45,Montana,46,1,0,0,montana,0.022222,NaN


# Q&A Scraping FAQ

In [229]:
from bs4 import BeautifulSoup
from requests import get
import difflib

In [629]:
def get_questions():
    who_q, who_a = get_who_questions()
    cdc_q, cdc_a = get_cdc_questions()
    fda_q, fda_a = get_fda_questions()
    jhm_q, jhm_a = get_jhm_questions()
    cnn_q, cnn_a = get_cnn_questions()
    questions = who_q + cdc_q + fda_q + jhm_q + cnn_q
    answers = who_a + cdc_a + fda_a + jhm_a + cnn_a
    print(len(answers), len(questions))
    return (questions, answers)

def get_cnn_questions():
    # Scraping from CNN FAQ
    url = 'https://www.cnn.com/interactive/2020/health/coronavirus-questions-answers/'
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    questions = html_soup.find_all('div', class_='question-q')
    questions = [q.text.strip().replace('\x99','').replace('\x80','').replace('â',"'") for q in questions]

    answers = html_soup.find_all('div', class_='question-a')
    answers = [a.text.strip().replace('\x99','').replace('\x80','').replace('\x93','').replace('\x94','').replace('â',"'").replace('\x9c','').replace('\x9d','') for a in answers]

    return (questions, answers)


def get_jhm_questions():
    # Scraping from Johns Hopkins Medicine
    url = 'https://www.hopkinsmedicine.org/health/conditions-and-diseases/coronavirus/coronavirus-frequently-asked-questions'
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    questions = []
    answers = []

    for header in html_soup.find_all('h3'):
        para = header.find_next_sibling('p')
        questions.append(header.text.strip())
        if para:
            answers.append(para.text.strip().replace('\n',' ').replace('\xa0',''))
            
    questions = questions[:-3]

    return (questions, answers)

def get_fda_questions():
    # Scraping from FDA
    url = 'https://www.fda.gov/emergency-preparedness-and-response/coronavirus-disease-2019-covid-19/coronavirus-disease-2019-covid-19-frequently-asked-questions'
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    questions = html_soup.find_all('div', class_ = 'panel-heading')
    questions = [q.text.strip().replace('Q: ','').replace('Q. ','') for q in questions]

    answers = html_soup.find_all('div', class_ = 'panel-body')
    answers = [a.text.strip().replace('A: ','').replace('A. ','').replace('\xa0', '').replace('A:','') for a in answers]

    return (questions, answers)


def get_cdc_questions():
    # Scraping from CDC
    url = 'https://www.cdc.gov/coronavirus/2019-ncov/faq.html'
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    questions = html_soup.find_all('button', class_ = 'card-title btn btn-link')
    questions = [q.text for q in questions]

    answers = html_soup.find_all('div', class_ = 'collapse')
    answers = [a.text for a in answers]
    answers.pop(0)

    return (questions, answers)

def get_who_questions():
    # Scraping from WHO
    url = 'https://www.who.int/news-room/q-a-detail/q-a-coronaviruses'
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    questions = html_soup.find_all('a', class_ = 'sf-accordion__link')
    questions = [' '.join(q.text.split()) for q in questions]

    answers = html_soup.find_all('p', class_ = 'sf-accordion__summary')
    answers = [' '.join(q.text.split()) for q in answers]

    return (questions, answers)

In [630]:
questions, answers = get_questions()

270 270


In [643]:
# Writing to a file
with open("corona-questions.txt", 'w') as f:
    for s in questions:
        f.write(str(s) + '`')
        
with open("corona-answers.txt", 'w') as f:
    for s in answers:
        f.write(str(s) + '`')

In [2]:
# Reading from a file
with open("corona-questions.txt", 'r') as f:
    mystring = f.read()
questions = mystring.split("`")

with open("corona-answers.txt", 'r') as f:
    mystring = f.read()
answers = mystring.split("`")

In [3]:
len(questions), len(answers)

(272, 272)

In [9]:
answers[8]

'No. Antibiotics do not work against viruses, they only work on bacterial infections. COVID-19 is caused by a virus, so antibiotics do not work. Antibiotics should not be used as a means of prevention or treatment of COVID-19. They should only be used as directed by a physician to treat a bacterial infection.'

In [ ]:
COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus began in Wuhan, China, in December 2019.




In [18]:
def shorten_sentence(sent):
    num_sentences = sent.count('.')
    ans = sent
    while len(ans) > 160:
        # print(len(ans), num_sentences)
        ans = '.'.join(sent.split('.')[:num_sentences])
        num_sentences -= 1
    return ans

In [21]:
[shorten_sentence(s) for s in answers]

['Coronaviruses are a large family of viruses which may cause illness in animals or humans',
 'COVID-19 is the infectious disease caused by the most recently discovered coronavirus',
 'The most common symptoms of COVID-19 are fever, tiredness, and dry cough',
 'People can catch COVID-19 from others who have the virus',
 '',
 'The risk depends on where you are - and more specifically, whether there is a COVID-19 outbreak unfolding there',
 'Illness due to COVID-19 infection is generally mild, especially for children and young adults',
 '',
 'No. Antibiotics do not work against viruses, they only work on bacterial infections. COVID-19 is caused by a virus, so antibiotics do not work',
 '',
 'Not yet. To date, there is no vaccine and no specific antiviral medicine to prevent or treat COVID-2019',
 'No',
 'Only wear a mask if you are ill with COVID-19 symptoms (especially coughing) or looking after someone who may have COVID-19',
 'Remember, a mask should only be used by health workers, ca

In [86]:
a = "Respiratory viruses are not known to be transmitted by blood transfusion, and there have been no reported cases of transfusion-transmitted coronavirus."

len(a)    

    

151

In [98]:
answers[n][:160]

'There are no nationwide shortages of food, although in some cases the inventory of certain foods at your grocery store might be temporarily low before stores ca'

In [94]:
n = 157
print(questions[n], '\n', answers[n])

len(answers[n])

Will there be food shortages? 
 There are no nationwide shortages of food, although in some cases the inventory of certain foods at your grocery store might be temporarily low before stores can restock. Food production and manufacturing are widely dispersed throughout the U.S. and there are currently no widespread disruptions reported in the supply chain.
FDA is closely monitoring the food supply chain for any shortages in collaboration with industry and our federal and state partners. We are in regular contact with food manufacturers and grocery stores.


528

In [96]:
index = 0
for s in answers:
    if (len(shorten_sentence(s)) == 0):
        print('\n')
        print(index)
        print(questions[index])
        print(s)
    index+=1
    



4
What can I do to protect myself and prevent the spread of disease?
Protection measures for everyoneStay aware of the latest information on the COVID-19 outbreak, available on the WHO website and through your national and local public health authority. Many countries around the world have seen cases of COVID-19 and several have seen outbreaks. Authorities in China and some other countries have succeeded in slowing or stopping their outbreaks. However, the situation is unpredictable so check regularly for the latest news.You can reduce your chances of being infected or spreading COVID-19 by taking some simple precautions:Regularly and thoroughly clean your hands with an alcohol-based hand rub or wash them with soap and water.Why? Washing your hands with soap and water or using alcohol-based hand rub kills viruses that may be on your hands.Maintain at least 1 metre (3 feet) distance between yourself and anyone who is coughing or sneezing.Why? When someone coughs or sneezes they spray 

# String Comparison

Using BOW (Bag of Words) and NLTK
[Link](https://towardsdatascience.com/calculating-string-similarity-in-python-276e18a7d33a)

Levenshtein [Link](https://pypi.org/project/python-Levenshtein/)

In [375]:
import Levenshtein as lev

import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
sw = stopwords.words('english')  

In [401]:
def clean_string(text):
    text = ''.join([word for word in text if word not in string.punctuation])
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in sw])
    return text

def cosine_sim_vectors(vec1, vec2):
    vec1 = vec1.reshape(1,-1)
    vec2 = vec2.reshape(1,-1)
    
    return cosine_similarity(vec1, vec2)[0][0]

In [543]:
cleaned = list(map(clean_string, questions))
vectoriser = CountVectorizer(cleaned)
question_vectors = vectoriser.fit_transform(cleaned[:]).toarray()
question_vectors.shape
# vectoriser.vocabulary_

(270, 614)

In [559]:
input_q = "should i wear a face mask"

ratios = [lev.ratio(input_q, q) for q in questions]

input_vector = vectoriser.transform([input_q])
cosine_sim = [cosine_sim_vectors(input_vector[0], q) for q in question_vectors]

max_value_c = max(cosine_sim)
max_index_c = cosine_sim.index(max_value_c)
max_value_r = max(ratios)
max_index_r = ratios.index(max_value_r)

print("cosine", max_value_c, questions[max_index_c])
print("\nratio", max_value_r, questions[max_index_r])


if max_value_c > max_value_r:
    max_value = max_value_c
    max_index = max_index_c
else:
    max_value = max_value_r
    max_index = max_index_r



ans = answers[max_index]
print("Max value and index", max_value, max_index)
print("Closest question:", questions[max_index])
print("Answer:", ans)

if max_value < 0.7:
    print("Please ask a question related to COVID-19. \nFor example: 'What is a coronavirus?'")

else:
    # Truncate to under 320 character (1 SMS is 160 characters)
    num_sentences = ans.count('.')
    while len(ans) > 320:
        # print(len(ans), num_sentences)
        ans = '.'.join(ans.split('.')[:num_sentences])
        num_sentences -= 1

    print(len(ans), num_sentences)

cosine 0.769800358919501 Should I wear a face covering or face mask when I go out in public?

ratio 0.8 Should I get a face mask?
Max value and index 0.769800358919501 115
Closest question: Should I wear a face covering or face mask when I go out in public?
Answer: CDC recommends wearing cloth face coverings in public when other social distancing measures are difficult to maintain (e.g., grocery stores and pharmacies) especially in areas of significant community-based transmission of the coronavirus. The purpose of wearing cloth face coverings in public is to slow the spread of the virus and help people who may have the virus and do not know it from transmitting it to others.
238 2


# Using Google Search API

In [14]:
# https://towardsdatascience.com/build-a-simple-chatbot-with-python-and-google-search-c000aa3f73f0

In [230]:
import requests
from lxml import html
from googlesearch import search
from bs4 import BeautifulSoup



In [231]:
query = "Should I wear a face mask for covid?"

## Google Search query results as a Python List of URLs
search_result_list = list(search(query, tld="co.in", num=10, stop=3, pause=1))

page = requests.get(search_result_list[0])

tree = html.fromstring(page.content)

soup = BeautifulSoup(page.content, features="lxml")


article_text = ''
article = soup.findAll('p')
for element in article:
    article_text += '\n' + ''.join(element.findAll(text = True))
article_text = article_text.replace('\n', '')
first_sentence = article_text.split('.')
first_sentence = first_sentence[0].split('?')[0]

print(first_sentence)

ByLaura Geggel - Associate Editor31 March 2020Wearing a face mask could protect you in crowded situations


In [711]:
# page.content